In [14]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray
import xarray.ufuncs as xu
import dask
import seaborn as sn
from datetime import datetime
from dask.diagnostics import ProgressBar
import dask
import warnings
import os
import scipy
from pytz import timezone
import re

In [ ]:
ds = xarray.open_dataset('/home/araldif/huckley/files/past_1980_Tsurface.nc')
ds.__delitem__('soil_mic')
ds.__delitem__('soil_wrf')
dsDates = ds.time.to_series()
subset = ds.sel(shade=0.0)
dsTsurf = xarray.Dataset({'Tsurf': (['time','south_north','west_east'], subset.Tsurface.values)}, coords = {'longitude': (['south_north','west_east'], ds.lon.values),
                                           'latitude': (['south_north','west_east'], ds.lat.values),
                                           'time': xarray.DataArray.from_series(dsDates).values, 'shade': ds.shade.values})

In [23]:
ds = xarray.open_dataset('/datadrive/huckley/files/new_1981_Tsurface.nc')

In [24]:
ds['time'] = ds['time'].astype(int)

In [25]:
ds['time'] = pd.to_datetime(ds['time'],format='%Y%m%d%H')

In [26]:
ds.to_netcdf('./files/1981_Tsurface_fixed.nc')

In [27]:
concatenated_files = xarray.open_mfdataset('/datadrive/huckley/files/*_Tsurface_fixed.nc', concat_dim='time')

In [28]:
concatenated_files.to_netcdf('./concatenate_test.nc')

In [29]:
concatenated_files

<xarray.Dataset>
Dimensions:      (shade: 5, south_north: 129, time: 26280, west_east: 199)
Coordinates:
    longitude    (south_north, west_east) float64 -131.4 -131.1 -130.8 ...
    latitude     (south_north, west_east) float64 12.36 12.45 12.54 12.64 ...
  * shade        (shade) float64 0.0 25.0 50.0 75.0 100.0
  * south_north  (south_north) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * west_east    (west_east) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
  * time         (time) datetime64[ns] 1980-01-01T01:00:00 ...
Data variables:
    Tsurf        (time, south_north, west_east) float64 nan nan nan nan nan ...

In [30]:
bb = {'minLong':-119.5, 'maxLong':-119.13, 'minLat': 37.66, 'maxLat':38.03}
long = concatenated_files.coords['longitude'].values
lat = concatenated_files.coords['latitude'].values

In [31]:
xycord = np.where( (long > bb['minLong'] ) & (long < bb['maxLong']) & (lat > bb['minLat']) & (lat < bb['maxLat']))
xcord = xycord[:][0]
ycord = xycord[:][1]

In [32]:
reduced_ds = concatenated_files.isel(south_north=xcord, west_east=ycord)

In [33]:
reduced_ds

<xarray.Dataset>
Dimensions:      (shade: 5, south_north: 1, time: 26280, west_east: 1)
Coordinates:
    longitude    (south_north, west_east) float64 -119.3
    latitude     (south_north, west_east) float64 37.85
  * shade        (shade) float64 0.0 25.0 50.0 75.0 100.0
  * south_north  (south_north) int64 68
  * west_east    (west_east) int64 53
  * time         (time) datetime64[ns] 1980-01-01T01:00:00 ...
Data variables:
    Tsurf        (time, south_north, west_east) float64 270.5 269.6 268.9 ...